<h2>This script is for internal developer testing.</h2>

In [ ]:
import numpy as np
from wfdb import readsignal
# Testing rdsamp format 212
sig, fields=readsignal.rdsamp('sampledata/100', sampfrom=100, sampto=9999) # Default physical values
print(sig)
print(len(sig))

sig, fields=readsignal.rdsamp('sampledata/100', physical=0) # Digital values
print(sig)

In [ ]:
# Testing rdsamp format 16
sig, fields=readsignal.rdsamp('sampledata/test01_00s')
print(sig)
print(fields)


In [ ]:
# Testing rdsamp format 80
sig, fields=readsignal.rdsamp('sampledata/3000003_0003')
print(sig)
print(fields)



In [1]:
# Testing rdsamp format 16 with byte offset (actually a .mat file too)
import numpy as np
from wfdb import readsignal
sig, fields=readsignal.rdsamp('sampledata/a103l') 
#print(sig)
#print(sig.shape)
#print(fields)

from wfdb import plotwfdb

plotwfdb.plotsigs(sig, fields)


# With channel selection and sampfrom sampto
#sig, fields=readsignal.rdsamp('sampledata/a103l', channels=[2,0], sampfrom=250, sampto=80000) 
#print('\n\n')
#print(sig)
#print(fields)

In [ ]:
# Testing rdsamp multi-segment variable layout record (from https://physionet.org/physiobank/database/mimic2wdb/matched/s00001/)
import numpy as np
from wfdb import readsignal

# Return list of arrays
sig, fields=readsignal.rdsamp('sampledata/multisegmentV/s00001-2896-10-10-00-31', sampfrom=3200, sampto=20000, stacksegments=0)
#print("len(sig): ", len(sig))
#print(len(fields))

print(sig)
print("\n\n")
print(sig[len(sig)-1])
print("\n\n")
print(len(sig[len(sig)-1]))
print("\n\n")
print(fields[0])
print("\n\n")
print(fields[1])
print("\n\n")
print(fields[2])

In [3]:
# Testing rdsamp multi-segment variable layout record - stacksegments
import numpy as np
from wfdb import readsignal


# Return single numpy array
sig, fields=readsignal.rdsamp('sampledata/multisegmentV/s00001-2896-10-10-00-31', stacksegments=1, sampto=10000, channels=[1, 2])

print(sig)
print(sig.shape)
print("\n\n")
print(fields[0])
print("\n\n")
print(fields[1])
print("\n\n")
print(fields[2])

[[ nan  nan]
 [ nan  nan]
 [ nan  nan]
 ..., 
 [-21.  nan]
 [-21.  nan]
 [-20.  nan]]
(10000, 2)



{'nsig': 4, 'basedate': '10/10/2896', 'gain': [], 'baseline': [], 'filename': ['3975656_layout', '~', '3975656_0001', '3975656_0002', '3975656_0003', '3975656_0004', '3975656_0005', '~', '3975656_0006', '3975656_0007', '3975656_0008', '3975656_0009', '~', '3975656_0010', '~', '3975656_0011', '3975656_0012', '3975656_0013', '3975656_0014', '3975656_0015', '3975656_0016'], 'nsamp': 14518365, 'fmt': [], 'comments': [' <age>: 60 <sex>: F'], 'byteoffset': [], 'fs': 125.0, 'units': [], 'signame': [], 'initvalue': [], 'nseg': 21, 'nsampseg': [0, 3261, 6750, 125, 1354, 512, 906259, 2604, 1304941, 1280, 512, 2210767, 172500, 5760000, 157500, 3900000, 4425, 18075, 7500, 37500, 22500], 'basetime': '00:31:25.894'}



{'nsig': 4, 'basedate': '', 'gain': [83.0, 55.0, 122.0, 1.0], 'baseline': [0, 0, 0, 0], 'filename': ['~', '~', '~', '~'], 'nsamp': 0, 'fmt': ['0', '0', '0', '0'], 'comments': [], 'byteo

In [5]:
# Testing rdsamp multi-dat signals (from https://physionet.org/physiobank/database/ptbdb/patient001/)
import numpy as np
from wfdb import readsignal

sig, fields=readsignal.rdsamp('sampledata/s0010_re', channels=[14, 0, 5, 10, 11], sampfrom=100, sampto=20000)
from wfdb import plotwfdb
plotwfdb.plotsigs(sig, fields)
print("\n\n\n", sig, "\n\n\n")
print(fields)
print(fields["signame"])




 [[ 0.0335 -0.167  -0.237   0.1165  0.1105]
 [ 0.0355 -0.1615 -0.2395  0.119   0.1135]
 [ 0.0385 -0.168  -0.2465  0.116   0.112 ]
 ..., 
 [-0.007   0.078   0.0445  0.008  -0.009 ]
 [-0.0065  0.068   0.0545  0.0125 -0.0055]
 [-0.005   0.058   0.061   0.022   0.0015]] 



{'nsig': 5, 'basedate': '', 'gain': [2000.0, 2000.0, 2000.0, 2000.0, 2000.0], 'baseline': [0, 0, 0, 0, 0], 'filename': ['s0010_re.xyz', 's0010_re.dat', 's0010_re.dat', 's0010_re.dat', 's0010_re.dat'], 'nsamp': 38400, 'fmt': ['16', '16', '16', '16', '16'], 'comments': [' age: 81', ' sex: female', ' ECG date: 01/10/1990', ' Diagnose:', ' Reason for admission: Myocardial infarction', ' Acute infarction (localization): infero-latera', ' Former infarction (localization): no', ' Additional diagnoses: Diabetes mellitu', ' Smoker: no', ' Number of coronary vessels involved: 1', ' Infarction date (acute): 29-Sep-90', ' Previous infarction (1) date: n/a', ' Previous infarction (2) date: n/a', ' Hemodynamics:', ' Catheterizatio

In [ ]:
# Testing rdsamp multi-segment record with sampfrom, sampto, and channel selection
import numpy as np
from wfdb import readsignal

# Return list of arrays
sig, fields=readsignal.rdsamp('sampledata/multisegmentV/s00001-2896-10-10-00-31', sampfrom=10, sampto=10200, channels=[2,3], stacksegments=0)

print(sig)
# Return single array
sig, fields=readsignal.rdsamp('sampledata/multisegmentV/s00001-2896-10-10-00-31', sampfrom=10, sampto=10200, channels=[1,0,2],stacksegments=1)

print(sig)